<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT License.</i>

<i> Created by Chenhui Hu </i>

# Model Tuning and  Deployment using Azure Machine Learning Service


In this notebook, we perform hyperparameter tuning of a LightGBM retail sales forecast model using HyperDrive in Azure Machine Learning (AzureML). After the optimal hyperparameters are found, we further deploy the best model as a web service on Azure.

To tune the hyperparameters, we carry out cross-validation with the Orange Juice data from week 40 to week 135. Specifically, we split the data into a training set and a validation set. Then, we train LightGBM models with different sets of hyperparameters on the training set and evaluate the accuracy of each model on the validation set. The set of hyperparameters which yield the best validation accuracy will be used to train forecast models when the data beyond week 135 is available, e.g., in the multi-round training examples provided in [examples/02_model](../02_model).

## Prerequisites

To run this notebook, you need to start from a conda environment where AzureML SDK is installed. In our case, we can first activate `forecasting_env` environment by
```
conda activate forecasting_env
```
as we have installed AzureML SDK in this environment. Then, we can start the notebook via
```
jupyter notebook --no-browswers
```
In addition, you need to install and enable AzureML widget extension in your environment by running the following commands.
```
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable --py --user azureml.widgets
```

Besides, you need to create an AzureML workspace and download its configuration file (`config.json`) by following the instructions in [configuration.ipynb](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook.

## Global Settings

In [1]:
%load_ext autoreload
%load_ext blackcellmagic
%autoreload 2

In [2]:
# Install and enable AzureML widgets
!jupyter nbextension install --py --user azureml.widgets
!jupyter nbextension enable --py --user azureml.widgets

Installing /data/anaconda/envs/forecasting_env/lib/python3.6/site-packages/azureml/widgets/static -> azureml_widgets
Up to date: /data/home/chenhui/.local/share/jupyter/nbextensions/azureml_widgets/index.js
Up to date: /data/home/chenhui/.local/share/jupyter/nbextensions/azureml_widgets/extension.js
Up to date: /data/home/chenhui/.local/share/jupyter/nbextensions/azureml_widgets/packages/labextension/azureml_widgets-1.1.0.tgz
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable azureml.widgets --user --py
    
Enabling notebook extension azureml_widgets/extension...
      - Validating: OK


In [3]:
import os
import sys
import json
import azureml
import requests
import subprocess
import numpy as np
from azureml.core import (
    Experiment,
    ScriptRunConfig,
)
from azureml.telemetry import set_diagnostics_collection
from azureml.core.runconfig import (
    RunConfiguration,
    EnvironmentDefinition,
    CondaDependencies,
)
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import (
    BayesianParameterSampling,
    HyperDriveConfig,
    quniform,
    uniform,
    choice,
    PrimaryMetricGoal,
)
from azureml.core.webservice import AciWebservice
from azureml.core.model import (
    Model,
    InferenceConfig,
)
from fclib.common.utils import git_repo_path
from fclib.azureml.azureml_utils import (
    get_or_create_workspace,
    get_or_create_amlcompute,
)
from fclib.dataset.ojdata import download_ojdata, split_train_test

cur_dir = os.getcwd()
if cur_dir not in sys.path:
    sys.path.append(cur_dir)
from aml_scripts.utils import create_features

# Opt-in diagnostics for better experience of future releases
set_diagnostics_collection(send_diagnostics=True)

# Check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Turning diagnostics collection on. 
Azure ML SDK Version:  1.0.85


In [4]:
# Use False if you've already downloaded and split the data
DOWNLOAD_SPLIT_DATA = False #True

# Get data directory
DATA_DIR = os.path.join(git_repo_path(), "ojdata")

# Forecasting settings
N_SPLITS = 1
HORIZON = 2
GAP = 2
FIRST_WEEK = 40
LAST_WEEK = 138

## Initialize Workspace & Create an AzureML Experiment

Initialize a [Machine Learning Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the workspace you created in the Prerequisites step. `get_or_create_workspace()` below creates a workspace object from the details stored in `config.json` that you have downloaded. We assume that you store this config file to a directory `./.azureml`. In case the existing workspace cannot be loaded, the following cell will try to create a new workspace with the subscription ID, resource group, and workspace name as specified in the beginning of the cell.

In [5]:
# Please specify the AzureML workspace attributes below if you want to create a new one.
subscription_id = "<subscription-id>"
resource_group = "<resource-group>"
workspace_name = "<workspace-name>"
workspace_region = "<workspace-region>"

# Connect to a workspace
ws = get_or_create_workspace(
    config_path="./.azureml",
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace_name=workspace_name,
    workspace_region=workspace_region,
)
print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

Workspace name: chhamlws
Azure region: westcentralus
Resource group: chhamlwsrg


In [6]:
# Create an experiment
exp = Experiment(workspace=ws, name="tune-lgbm-forecast")

## Data Preparation

We need to download the Orange Juice data and split it into training and test sets. By default, the following cell will download and spit the data. If you've already done so, you may skip this part by switching `DOWNLOAD_SPLIT_DATA` to False. 

By passing `write_csv=True` to `split_train_test()` below, this function will write the training data and test data to three csv files: `train.csv`, `auxi.csv` and `test.csv`. The first two csv files contain the historical sales up to week 135 as well as auxiliary information such as future price and promotion. Here we assume that future price and promotion information up to a certain number of weeks ahead is predetermined and known. We will use these two files to implement cross-validation and search for the best model with HyperDrive.

In [7]:
if DOWNLOAD_SPLIT_DATA:
    download_ojdata(DATA_DIR)
    split_train_test(
        DATA_DIR,
        n_splits=N_SPLITS,
        horizon=HORIZON,
        gap=GAP,
        first_week=FIRST_WEEK,
        last_week=LAST_WEEK,
        write_csv=True,
    )

## Validate Script Locally

A good practice is to test the model training and validation script on your local machine before you run the hyperparameter tuning job on a remote compute. To run the script locally, we need to correctly specify the path of the Python interpreter that has been installed in `forecasting_env` conda environment. In what follows, the script `train_validate.py` trains a model on the training set with the input arguments as specified in `ScriptRunConfig()` and computes the accuracy of the model on the validation set. Here we evaluate the model accuracy using mean-absolute-percentage-error (MAPE).

In [8]:
# Get Python interpreter path
python_path = subprocess.check_output("which python", shell=True)
python_path = python_path.decode("utf-8")[:-1]

# Configure local, user managed environment
run_config_user_managed = RunConfiguration()
run_config_user_managed.environment.python.user_managed_dependencies = True
run_config_user_managed.environment.python.interpreter_path = python_path

In [9]:
# Directory of the local scripts
script_folder = "./aml_scripts"

# Training script name and path
train_script_name = "train_validate.py"
train_script_path = os.path.join(script_folder, train_script_name)

# Specify script run config
src = ScriptRunConfig(
    source_directory="./",
    script=train_script_path,
    arguments=["--data-folder", DATA_DIR, "--bagging-fraction", "0.8"],
    run_config=run_config_user_managed,
)
run_local = exp.submit(src)

In [10]:
# Check job status
run_local.get_status()

'Running'

We will wait until the local run finishes. Then, we print out the validation metric. Moreover, you can also use `run_local.get_details()` to get detailed information about this run.

In [11]:
# Check results
while run_local.get_status() != "Completed":
    {}
run_local.get_metrics()

{'MAPE': 63.79837613183825}

## Run Script on Remote Compute

After validating model training script locally, we can create a remote compute and further test the script on the remote compute.

### Create a CPU cluster as compute target

In the next cell, we create an AmlCompute target with a specific cluster name, VM size, and maximum number of nodes if the cluster does not exist. Otherwise, we will reuse an existing one. For more options of VM sizes, you can check information in this [link](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general).

In [12]:
# Choose a name for your cluster
cluster_name = "cpu-cluster"
# VM Size
vm_size = "STANDARD_D2_V2"
# Maximum number of nodes of the cluster
max_nodes = 4

# Create a new AmlCompute if it does not exist or reuse an existing one
compute_target = get_or_create_amlcompute(
    workspace=ws,
    compute_name=cluster_name,
    vm_size=vm_size,
    min_nodes=0,
    max_nodes=max_nodes,
    verbose=True,
)

Found compute target: cpu-cluster


### Configure Docker environment

The remote compute will need to create a [Docker image](https://docs.docker.com/get-started/) for running the script. The Docker image is an encapsulated environment with necessary dependencies installed. In the following cell, we specify the conda packages and Python version that are needed for running the script.

In [13]:
env = EnvironmentDefinition()
env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "numpy", "scipy", "scikit-learn", "lightgbm", "joblib"],
    python_version="3.6.2",
)
env.python.conda_dependencies.add_channel("conda-forge")
env.docker.enabled = True

### Upload data to default datastore

Each workspace comes with a default datastore. In the following, we upload the Orange Juice dataset to the workspace's default datastore, which will later be mounted on the cluster for model training and validation.

In [14]:
ds = ws.get_default_datastore()
print(
    "Datastore type: " + ds.datastore_type,
    "Account name: " + ds.account_name,
    "Container name: " + ds.container_name,
    sep="\n",
)

Datastore type: AzureBlob
Account name: chhamlws4931040064
Container name: azureml-blobstore-f799a640-1ca3-4877-ad24-08eef7bd307e


In [15]:
# Remote data path
path_on_datastore = "data"
ds.upload(
    src_dir=DATA_DIR,
    target_path=path_on_datastore,
    overwrite=True,
    show_progress=False,
)

$AZUREML_DATAREFERENCE_cf8c449e8b1f468fa912bdeb311cd90a

In [16]:
# Get data reference object for the data path
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_f9d2a744355a4ce3aadabbb8697ed14a


### Create estimator

Next, we will check if the remote compute target is successfully created by submitting a job to the target. This compute target will be used by HyperDrive for hyperparameter tuning later. Note that you may skip this part and directly go to [Tune Hyperparameters using HyperDrive](#tune-hyperparameters-using-hyperdrive) if you want.

In the following cells, we first create an estimator to specify details of the job. Then we sumbit the job to the remote compute and check the status of the job.

In [17]:
script_params = {"--data-folder": ds_data.as_mount(), "--bagging-fraction": 0.8}
est = Estimator(
    source_directory=script_folder,
    script_params=script_params,
    compute_target=compute_target,
    use_docker=True,
    entry_script=train_script_name,
    environment_definition=env,
)

### Submit job

In [18]:
# Submit job to remote compute
run_remote = exp.submit(config=est)

### Check job status

You can monitor the status of the remote run using the AzureML widgets. After the job is done, the following cell will display a dashboard similar as

<img src="https://user-images.githubusercontent.com/20047467/76150936-67fc7c00-607d-11ea-9354-418bd5f733d6.png" width="900" height="360">

In [19]:
RunDetails(run_remote).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

We can check the validation metric after the job finishes. The validation metric should be the same as the one we obtained when the script was ran locally. For more details of the job, you can execute `run_remote.get_details()`.

In [20]:
# Get metric value after the job finishes
while run_remote.get_status() != "Completed":
    {}
run_remote.get_metrics()

{'MAPE': 63.79837613183825}

<a id='tune-hyperparameters-using-hyperdrive'></a>
## Tune Hyperparameters using HyperDrive

Now we are ready to tune the hyperparameters of the LightGBM forecast model by launching multiple runs on the cluster. In the following cell, we define the configurations of a HyperDrive job that does a parallel searching of the hyperparameter space using a Bayesian sampling method. HyperDrive also supports random sampling of the parameter space.

It is recommended that the maximum number of runs should be greater than or equal to 20 times the number of hyperparameters being tuned, for best results with Bayesian sampling. Specifically, it should be no less than 180 in the following case. Nevertheless, we find that even with very small amount of runs Bayesian search can achieve decent performance. Thus, the maximum number of child runs of HyperDrive `max_total_runs` is set as `20` to reduce the running time.

In [21]:
# Increase this value if you want to achieve better performance
max_total_runs = 20
script_params = {"--data-folder": ds_data.as_mount()}
est = Estimator(
    source_directory=script_folder,
    script_params=script_params,
    compute_target=compute_target,
    use_docker=True,
    entry_script=train_script_name,
    environment_definition=env,
)

# Specify hyperparameter space
ps = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.2, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 20, 1),
        "--max-rounds": quniform(50, 2000, 10),
        "--max-lag": quniform(3, 40, 1),
        "--window-size": quniform(3, 40, 1),
    }
)

# HyperDrive job configuration
htc = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name="MAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=max_total_runs,
    max_concurrent_runs=4,
)

htr = exp.submit(config=htc)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Current value for max_total_runs:20. Recommendend value:180.


After the job finishes, you should see outputs from the AzureML widgets similar to the following. Note that you can rerun  `RunDetails(htr).show()` after the job finishes to get the updated results on the dashboard in case it is not automatically refreshed.

<img src="https://user-images.githubusercontent.com/20047467/76152567-e31a5e00-608e-11ea-90a8-3fdfeafeeb92.png" width="900" height="500">

<img src="https://user-images.githubusercontent.com/20047467/76152586-270d6300-608f-11ea-8f83-07fa7a8528f2.png" width="900" height="400">

<img src="https://user-images.githubusercontent.com/20047467/76152600-46a48b80-608f-11ea-90d2-297aabd7376a.png" width="900" height="300">

In [22]:
RunDetails(htr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

In [23]:
while htr.get_status() != "Completed":
    {}
htr.get_metrics()

{'HD_9d84746f-6a79-4a7a-908e-d87b22964781_0': {'MAPE': 36.14593979500759},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_1': {'MAPE': 51.207370474510164},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_10': {'MAPE': 32.446632612512},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_11': {'MAPE': 42.02577284190877},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_12': {'MAPE': 35.61336578834721},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_13': {'MAPE': 73.79327663085509},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_14': {'MAPE': 73.37497628117707},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_15': {'MAPE': 33.68597795822114},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_16': {'MAPE': 31.91582713727422},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_17': {'MAPE': 40.2927561100981},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_18': {'MAPE': 41.11747407801938},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_19': {'MAPE': 33.17396641669122},
 'HD_9d84746f-6a79-4a7a-908e-d87b22964781_2': {'MAPE': 30.954749012270483},
 'HD_9d84746f-6a

The best model and its hyperparameter values can be retrieved as follows

In [24]:
best_run = htr.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()["runDefinition"]["arguments"]
print(parameter_values)

['--data-folder', '$AZUREML_DATAREFERENCE_f9d2a744355a4ce3aadabbb8697ed14a', '--num-leaves', '116', '--min-data-in-leaf', '340', '--learning-rate', '0.03', '--feature-fraction', '0.730454396811419', '--bagging-fraction', '0.996974494629062', '--bagging-freq', '8', '--max-rounds', '1360', '--max-lag', '38', '--window-size', '23']


We can then register the folder (and all files in it) as a model named `lgbm-oj-forecast` under the workspace for deployment.

In [25]:
model = best_run.register_model(
    model_name="lgbm-oj-forecast", model_path="outputs/model"
)

## Deploy the Model in ACI

Now we are ready to deploy the model as a web service running in Azure Container Instance [ACI](https://azure.microsoft.com/en-us/services/container-instances/). Azure Machine Learning accomplishes this by constructing a Docker image with the scoring logic and model baked in.

### Create score.py

First, we will create a scoring script that will be invoked by the web service call.

* Note that the scoring script must have two required functions, `init()` and `run(input_data)`.
    - In `init()` function, you typically load the model into a global object. This function is executed only once when the Docker container is started.
    - In `run(input_data)` function, the model is used to predict a value based on the input data. The input and output to run typically use JSON as serialization and de-serialization format but you are not limited to that.

In [26]:
%%writefile score.py
import os
import json
import numpy as np
import pandas as pd
import lightgbm as lgb


def init():
    global bst
    model_root = os.getenv("AZUREML_MODEL_DIR")
    # The name of the folder in which to look for LightGBM model files
    lgbm_model_folder = "model"
    bst = lgb.Booster(
        model_file=os.path.join(model_root, lgbm_model_folder, "bst-model.txt")
    )


def run(raw_data):
    columns = bst.feature_name()
    data = np.array(json.loads(raw_data)["data"])
    test_df = pd.DataFrame(data=data, columns=columns)
    # Make prediction
    out = bst.predict(test_df)
    return out.tolist()

Overwriting score.py


### Create myenv.yml

We also need to create an environment file so that Azure Machine Learning can install the necessary packages in the Docker image which are required by your scoring script. In this case, we need to specify packages `numpy`, `pandas`, and `lightgbm`.

In [27]:
cd = CondaDependencies.create()
cd.add_conda_package("numpy=1.16.2")
cd.add_conda_package("pandas=0.23.4")
cd.add_conda_package("lightgbm=2.3.0")
cd.save_to_file(base_directory="./", conda_file_path="myenv.yml")

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
- numpy=1.16.2
- pandas=0.23.4
- lightgbm=2.3.0
channels:
- conda-forge



### Deploy to ACI

We are almost ready to deploy. In the next cell, we first create the inference configuration and deployment configuration. Then, we deploy the model to ACI. This cell will run for several minutes.

In [28]:
%%time

inference_config = InferenceConfig(runtime="python", entry_script="score.py", conda_file="myenv.yml")

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    tags={"name": "ojdata", "framework": "LightGBM"},
    description="LightGBM model on Orange Juice data",
)

service = Model.deploy(
    workspace=ws, name="lgbm-oj-svc", models=[model], inference_config=inference_config, deployment_config=aciconfig
)

service.wait_for_deployment(True)
print(service.state)

Running.............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 568 ms, sys: 141 ms, total: 709 ms
Wall time: 2min 46s


> Tip: If something goes wrong with the deployment, you could look at the logs from the service by running this command `print(service.get_logs())`.

This is the scoring web service endpoint:

In [29]:
print(service.scoring_uri)

http://1df089bb-be2f-47bb-8f62-fa30f476fe43.westus.azurecontainer.io/score


After the web service is successfully deployed, you will see a deployment in the Azure Machine Learning workspace on Azure portal

<img src="https://user-images.githubusercontent.com/20047467/76572336-2100f300-6490-11ea-9467-83cd693d23c1.png" width="900" height="500">

### Test the deployed model

Let's test the deployed model. We create a few test data points and send them to the web service hosted in ACI. Note here we are using the run API in the SDK to invoke the service. You can also make raw HTTP calls using any HTTP tool such as curl.

After the invocation, we print the returned predictions each of which represents the forecasted sales of a target store, brand in a given week as specified by `store, brand, week` in `used_columns`.

In [30]:
# Prepare features according to the input schema of the best model
train_dir = os.path.join(DATA_DIR, "train")
max_lag = int(parameter_values[parameter_values.index("--max-lag") + 1])
lags = np.arange(2, max_lag + 1)
window_size = int(parameter_values[parameter_values.index("--window-size") + 1])
used_columns = [
    "store",
    "brand",
    "week",
    "week_of_month",
    "month",
    "deal",
    "feat",
    "move",
    "price",
    "price_ratio",
]
GAP = 2
features, train_end_week = create_features(
    1, train_dir, lags, window_size, used_columns
)
test_fea = features[features.week >= train_end_week + GAP].reset_index(drop=True)
test_fea.drop("move", axis=1, inplace=True)

# Pick a few test data points
test_samples = json.dumps({"data": np.array(test_fea.iloc[:3]).tolist()})
test_samples = bytes(test_samples, encoding="utf8")

# Predict using the deployed model
result = service.run(input_data=test_samples)
print("prediction:", result)

prediction: [10865.700075504421, 20498.24318392933, 7172.911679596602]


We can also send raw HTTP request to the service.

In [31]:
headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, test_samples, headers=headers)

print("POST to url", service.scoring_uri)
print("")
print("input data:", test_samples)
print("")
print("prediction:", resp.text)

POST to url http://1df089bb-be2f-47bb-8f62-fa30f476fe43.westus.azurecontainer.io/score

input data: b'{"data": [[2.0, 1.0, 137.0, 4.0, 4.0, 0.0, 0.0, 0.0416446872, 1.1124927835293534, 12416.0, 28096.0, 15168.0, 20736.0, 31808.0, 25728.0, 43584.0, 5056.0, 20224.0, 6720.0, 5504.0, 3520.0, 9792.0, 13120.0, 13120.0, 17728.0, 8320.0, 5120.0, 6080.0, 7168.0, 34240.0, 7296.0, 9216.0, 5312.0, 6272.0, 6784.0, 6784.0, 7744.0, 6016.0, 9024.0, 5632.0, 39424.0, 4160.0, 13568.0, 13568.0, 13568.0, 21952.0, 15206.95652173913], [2.0, 1.0, 138.0, 5.0, 4.0, 1.0, 1.0, 0.03734375, 0.9420125411290402, 12416.0, 12416.0, 28096.0, 15168.0, 20736.0, 31808.0, 25728.0, 43584.0, 5056.0, 20224.0, 6720.0, 5504.0, 3520.0, 9792.0, 13120.0, 13120.0, 17728.0, 8320.0, 5120.0, 6080.0, 7168.0, 34240.0, 7296.0, 9216.0, 5312.0, 6272.0, 6784.0, 6784.0, 7744.0, 6016.0, 9024.0, 5632.0, 39424.0, 4160.0, 13568.0, 13568.0, 13568.0, 15346.08695652174], [5.0, 1.0, 137.0, 4.0, 4.0, 0.0, 0.0, 0.04671875, 1.3116650049850451, 14144.0, 1

### Clean up

After finishing the tests, you can delete the ACI deployment with a simple delete API call as follows.

In [32]:
service.delete()

## Additional Reading:

\[1\] Training, hyperparameter tune, and deploy with TensorFlow: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/tensorflow/deployment/train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb <br>

\[2\] AzureML HyperDrive package: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py